# Comparing Dynawo vs. Astre results

A simple initial notebook to explore different ways in which to compare results, mostly graphically.  
(c) 2020 Grupo AIA
marinjl@aia.es


In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import sparklines
import os

os.getcwd()

'/home/marinjl/work'

In [2]:
#Case = namedtuple('Case', 'dynawo astre')
file_list = ['IEEE/IEEE14_DisconnectGroup/ORIG_CASE',
             'IEEE/IEEE14_DisconnectGroup/GEN1',
             'IEEE/IEEE14_DisconnectGroup/GEN3',
             'IEEE/IEEE14_DisconnectGroup/GEN6',
             'IEEE/IEEE14_DisconnectGroup/GEN8',
             'PtFige-Lille/20190410_1200/ORIG_CASE',
             'PtFige-Lille/20190410_1200/tFin_a',
             'PtFige-Lille/20190410_1200/tFin_b'
            ]

casenum = 4
dynawo_file = file_list[casenum] + "/outputs/curves/curves.csv"
astre_file  = file_list[casenum] + "/Astre/donneesModelesSortie.csv"
dict_file   = file_list[casenum] + "/../varnames_dict.txt"
html_output_file   = file_list[casenum] + "/gcompare"

dynawo_file

'IEEE/IEEE14_DisconnectGroup/GEN8/outputs/curves/curves.csv'

### Read dynawo output

In [3]:
cols = pd.read_csv(dynawo_file, sep=";", nrows=1)
cols = cols.columns.tolist()
cols_to_use = cols[:-1] # Do not read last col, it's empty (the files contain an extra ; at the end)
dydata = pd.read_csv(dynawo_file, sep=";", usecols=cols_to_use).T
# Make the index labels a column, for easier manipulation of columns later on:
dydata.reset_index(level=0, inplace=True)
dydata.rename(columns={"index": "variable"}, inplace=True)

In [4]:
dydata

variable           0           1           2  \
0                                 time    0.000000    0.000001    0.000002   
1       NETWORK__BUS____1_TN_Upu_value    1.059970    1.059970    1.059970   
2       NETWORK__BUS____2_TN_Upu_value    1.045041    1.045041    1.045041   
3       NETWORK__BUS____3_TN_Upu_value    1.010001    1.010001    1.010001   
4       NETWORK__BUS____4_TN_Upu_value    1.017703    1.017703    1.017703   
5       NETWORK__BUS____5_TN_Upu_value    1.019553    1.019553    1.019553   
6       NETWORK__BUS____6_TN_Upu_value    1.070149    1.070149    1.070149   
7       NETWORK__BUS____7_TN_Upu_value    1.061530    1.061530    1.061530   
8       NETWORK__BUS____8_TN_Upu_value    1.089926    1.089926    1.089926   
9       NETWORK__BUS____9_TN_Upu_value    1.055982    1.055982    1.055982   
10      NETWORK__BUS___10_TN_Upu_value    1.051051    1.051051    1.051051   
11      NETWORK__BUS___11_TN_Upu_value    1.057013    1.057013    1.057013   
12      NETWORK__BUS___12_TN_Upu_value    1.055333    1.055333    1.055333   
13      NETWORK__BUS___13_TN_Upu_value    1.050525    1.050525    1.050525   
14      NETWORK__BUS___14_TN_Upu_value    1.035657    1.035657    1.035657   
15       GEN____1_SM_generator_omegaPu    1.000000    1.000000    1.000000   
16          GEN____1_SM_generator_PGen  232.329584  232.329575  232.329575   
17          GEN____1_SM_generator_QGen  -16.691112  -16.691133  -16.691133   
18     GEN____1_SM_generator_UStatorPu    1.058824    1.058824    1.058824   
19  GEN____1_SM_voltageRegulator_EfdPu    1.436651    1.436651    1.436651   
20       GEN____2_SM_generator_omegaPu    1.000000    1.000000    1.000000   
21          GEN____2_SM_generator_PGen   40.126172   40.126182   40.126181   
22          GEN____2_SM_generator_QGen   43.675435   43.675431   43.675431   
23     GEN____2_SM_generator_UStatorPu    1.048778    1.048778    1.048778   
24  GEN____2_SM_voltageRegulator_EfdPu    1.269364    1.269364    1.269364   
25       GEN____3_SM_generator_omegaPu    1.000000    1.000000    1.000000   
26          GEN____3_SM_generator_PGen   -0.074832   -0.074841   -0.074841   
27          GEN____3_SM_generator_QGen   25.065946   25.065935   25.065935   
28     GEN____3_SM_generator_UStatorPu    1.011505    1.011505    1.011505   
29  GEN____3_SM_voltageRegulator_EfdPu    1.112250    1.112250    1.112250   
30       GEN____6_SM_generator_omegaPu    1.000000    1.000000    1.000000   
31          GEN____6_SM_generator_PGen    0.005609    0.005629    0.005629   
32          GEN____6_SM_generator_QGen   12.802790   12.802836   12.802836   
33     GEN____6_SM_generator_UStatorPu    1.070149    1.070149    1.070149   
34  GEN____6_SM_voltageRegulator_EfdPu    1.461749    1.461748    1.461748   
35       GEN____8_SM_generator_omegaPu    1.000000    1.000000    1.000000   
36          GEN____8_SM_generator_PGen   -0.006407   -0.006410   -0.006410   
37          GEN____8_SM_generator_QGen   17.569940   17.569931   17.569931   
38     GEN____8_SM_generator_UStatorPu    1.096374    1.096374    1.096374   
39  GEN____8_SM_voltageRegulator_EfdPu    1.194231    1.194231    1.194231   
40               _LOAD___2_EC_load_PPu    0.216990    0.216990    0.216990   
41               _LOAD___2_EC_load_QPu    0.126990    0.126990    0.126990   

             3           4           5           6           7           8  \
0     0.000004    0.000008    0.000016    0.000032    0.000064    0.000128   
1     1.059970    1.059970    1.059970    1.059970    1.059970    1.059970   
2     1.045041    1.045041    1.045041    1.045041    1.045041    1.045041   
3     1.010001    1.010001    1.010001    1.010001    1.010001    1.010001   
4     1.017703    1.017703    1.017703    1.017703    1.017703    1.017703   
5     1.019553    1.019553    1.019553    1.019553    1.019553    1.019553   
6     1.070149    1.070149    1.070149    1.070149    1.070149    1.070149   
7     1.061530    1.061530    1.061530    1.061530 

### Create and show sparklines of Dynawo results

In [5]:
# Insert sparklines in the same dataframe
dydata['Dynawo_result'] = sparklines.create(data=dydata.iloc[:,1:],
                                           figsize=(6, 0.3))
# Show them (in notebook):
sparklines.show(dydata[['variable', 'Dynawo_result']])

### Read Astre results

In [6]:
asdata = pd.read_csv(astre_file, sep=";").T
# Make the index labels a column, for easier manipulation of columns later on:
asdata.reset_index(level=0, inplace=True)
asdata.rename(columns={"index": "variable"}, inplace=True)

In [7]:
asdata

variable             0             1             2  \
0                             time      0.000000     10.000000     10.000000   
1         NOEUD_TENSION_BUS6VL/1.1     14.766000     14.766000     14.731700   
2          GRP_TENSION_STATOR_GEN6      1.070000      1.070000      1.067520   
3               GRP_PUISS_ACT_GEN6      0.010000      0.009987     -0.024449   
4               GRP_PUISS_REA_GEN6     12.729999     12.728700     18.640499   
5       GRP_TENSION_STATOR_KV_GEN6     14.766000     14.766000     14.731700   
6       NOEUD_TENSION_BUS 10VL/1.1     14.503600     14.503600     14.301900   
7       NOEUD_TENSION_BUS 11VL/1.1     14.585300     14.585300     14.466100   
8       NOEUD_TENSION_BUS 12VL/1.1     14.561600     14.561600     14.513100   
9       NOEUD_TENSION_BUS 13VL/1.1     14.495300     14.495300     14.433100   
10      NOEUD_TENSION_BUS 14VL/1.1     14.290300     14.290300     14.130700   
11        NOEUD_TENSION_BUS1VL/1.1     73.139999     73.139999     73.113701   
12         GRP_TENSION_STATOR_GEN1      1.058870      1.058870      1.058680   
13              GRP_PUISS_ACT_GEN1    232.348007    232.350006    231.789993   
14              GRP_PUISS_REA_GEN1    -16.542700    -16.542200    -13.965800   
15      GRP_TENSION_STATOR_KV_GEN1     25.412800     25.412800     25.408400   
16        NOEUD_TENSION_BUS2VL/1.1     72.105003     72.105003     72.049004   
17         GRP_TENSION_STATOR_GEN2      1.048730      1.048730      1.048280   
18              GRP_PUISS_ACT_GEN2     40.014099     40.009800     39.492500   
19              GRP_PUISS_REA_GEN2     43.543900     43.542400     47.803799   
20      GRP_TENSION_STATOR_KV_GEN2     25.169399     25.169399     25.158800   
21        NOEUD_TENSION_BUS3VL/1.1     69.690002     69.690002     69.653503   
22         GRP_TENSION_STATOR_GEN3      1.011500      1.011500      1.011200   
23              GRP_PUISS_ACT_GEN3      0.000000      0.009738     -0.748705   
24              GRP_PUISS_REA_GEN3     25.064199     25.068899     28.823999   
25      GRP_TENSION_STATOR_KV_GEN3     20.230101     20.230101     20.224001   
26        NOEUD_TENSION_BUS4VL/1.1     70.219597     70.219597     69.801399   
27        NOEUD_TENSION_BUS5VL/1.1     70.346703     70.346703     70.056396   
28        NOEUD_TENSION_BUS7VL/1.1     14.649000     14.649000     14.302200   
29        NOEUD_TENSION_BUS8VL/1.1     15.042000     15.042000     14.302200   
..                             ...           ...           ...           ...   
34      GRP_TENSION_STATOR_KV_GEN8     15.042000     15.042000     14.302200   
35   QUAD_PUISS_ACT_EX_BUS5-BUS6-1    -44.078800    -44.078201    -44.692402   
36      QUAD_INTENS_EX_BUS5-BUS6-1   1752.000000   1751.969970   1773.469970   
37  QUAD_REGLEUR_PRISE_BUS5-BUS6-1      0.000000      0.000000      0.000000   
38   QUAD_TD_DEPHASAGE_BUS5-BUS6-1      0.000000      0.000000      0.000000   
39   QUAD_PUISS_ACT_OR_BUS5-BUS6-1     44.078800     44.078201     44.692402   
40      QUAD_INTENS_OR_BUS5-BUS6-1    375.964996    375.959015    380.571991   
41    QUAD_TD_POSITION_BUS5-BUS6-1      0.000000      0.000000      0.000000   
42   QUAD_PUISS_ACT_OR_BUS2-BUS4-1     56.125099     56.124901     55.504902   
43   QUAD_PUISS_ACT_EX_BUS2-BUS4-1    -54.448799    -54.448700    -53.856499   
44      QUAD_INTENS_OR_BUS2-BUS4-1    449.569000    449.567993    444.964996   
45      QUAD_INTENS_EX_BUS2-BUS4-1    448.369995    448.368988    445.467987   
46   QUAD_TD_DEPHASAGE_BUS2-BUS4-1      0.000000      0.000000      0.000000   
47    QUAD_TD_POSITION_BUS2-BUS4-1      0.000000      0.000000      0.000000   
48     NOEUD_TENSION_PU_BUS3VL/1.1      1.010000      1.010000      1.009470   
49     GRST_CONSIGNE_REAC_APR_GEN3      0.000000      0.000000      0.000000   
50     GRST_SIGNAL_ERREUR_APR_GEN3      0.000000      0.000000      0.000000   
51     GRST_CONSIGNE_REAC_APR_GEN6      0.000000      0.000000      0.000000   
52     GRST_SIGNAL_ERREUR_

### Create and show sparklines of Astre results

In [8]:
# Insert sparklines in the same dataframe
asdata['Astre_result'] = sparklines.create(data=asdata.iloc[:,1:],
                                          color='#1b470a',
                                          fill_color='#99a894',
                                          fill_alpha=0.2,
                                          figsize=(6, 0.3))
# Show them (in notebook):
sparklines.show(asdata[['variable', 'Astre_result']])

### Compare the variables from one and the other

In [9]:
# Read the "dictionary" that translates var names
vardict = pd.read_csv(dict_file, sep=";", skipinitialspace=True, comment='#')
vardict

Dynawo                       Astre
0                                 time                        time
1        GEN____1_SM_generator_omegaPu                         NaN
2           GEN____1_SM_generator_PGen          GRP_PUISS_ACT_GEN1
3           GEN____1_SM_generator_QGen          GRP_PUISS_REA_GEN1
4      GEN____1_SM_generator_UStatorPu     GRP_TENSION_STATOR_GEN1
5   GEN____1_SM_voltageRegulator_EfdPu   GRP_CONSIGNE_TENSION_GEN1
6        GEN____2_SM_generator_omegaPu                         NaN
7           GEN____2_SM_generator_PGen          GRP_PUISS_ACT_GEN2
8           GEN____2_SM_generator_QGen          GRP_PUISS_REA_GEN2
9      GEN____2_SM_generator_UStatorPu     GRP_TENSION_STATOR_GEN2
10  GEN____2_SM_voltageRegulator_EfdPu                         NaN
11       GEN____3_SM_generator_omegaPu                         NaN
12          GEN____3_SM_generator_PGen          GRP_PUISS_ACT_GEN3
13          GEN____3_SM_generator_QGen          GRP_PUISS_REA_GEN3
14     GEN____3_SM_generator_UStatorPu     GRP_TENSION_STATOR_GEN3
15  GEN____3_SM_voltageRegulator_EfdPu                         NaN
16       GEN____6_SM_generator_omegaPu                         NaN
17          GEN____6_SM_generator_PGen          GRP_PUISS_ACT_GEN6
18          GEN____6_SM_generator_QGen          GRP_PUISS_REA_GEN6
19     GEN____6_SM_generator_UStatorPu     GRP_TENSION_STATOR_GEN6
20  GEN____6_SM_voltageRegulator_EfdPu                         NaN
21       GEN____8_SM_generator_omegaPu                         NaN
22          GEN____8_SM_generator_PGen          GRP_PUISS_ACT_GEN8
23          GEN____8_SM_generator_QGen          GRP_PUISS_REA_GEN8
24     GEN____8_SM_generator_UStatorPu     GRP_TENSION_STATOR_GEN8
25  GEN____8_SM_voltageRegulator_EfdPu                         NaN
26               _LOAD___2_EC_load_PPu                         NaN
27               _LOAD___2_EC_load_QPu                         NaN
28      NETWORK__BUS___10_TN_Upu_value  NOEUD_TENSION_BUS 10VL/1.1
29      NETWORK__BUS___11_TN_Upu_value  NOEUD_TENSION_BUS 11VL/1.1
30      NETWORK__BUS___12_TN_Upu_value  NOEUD_TENSION_BUS 12VL/1.1
31      NETWORK__BUS___13_TN_Upu_value  NOEUD_TENSION_BUS 13VL/1.1
32      NETWORK__BUS___14_TN_Upu_value  NOEUD_TENSION_BUS 14VL/1.1
33      NETWORK__BUS____1_TN_Upu_value    NOEUD_TENSION_BUS1VL/1.1
34      NETWORK__BUS____2_TN_Upu_value    NOEUD_TENSION_BUS2VL/1.1
35      NETWORK__BUS____3_TN_Upu_value    NOEUD_TENSION_BUS3VL/1.1
36      NETWORK__BUS____4_TN_Upu_value    NOEUD_TENSION_BUS4VL/1.1
37      NETWORK__BUS____5_TN_Upu_value    NOEUD_TENSION_BUS5VL/1.1
38      NETWORK__BUS____6_TN_Upu_value    NOEUD_TENSION_BUS6VL/1.1
39      NETWORK__BUS____7_TN_Upu_value    NOEUD_TENSION_BUS7VL/1.1
40      NETWORK__BUS____8_TN_Upu_value    NOEUD_TENSION_BUS8VL/1.1
41      NETWORK__BUS____9_TN_Upu_value    NOEUD_TENSION_BUS9VL/1.1

In [10]:
# Find the correspondences between Dynawo vars and Aster vars:
for vd in dydata['variable']:
    va = vardict.loc[vardict['Dynawo'] == vd, ['Astre']]
    #print("vd ==> %s; va ==> %s" % (vd, va))
    if (not va.empty and not va.isnull().values.any()):
        print("%s ==> %s" % (vd, va.iat[0,0]))


time ==> time
NETWORK__BUS____1_TN_Upu_value ==> NOEUD_TENSION_BUS1VL/1.1
NETWORK__BUS____2_TN_Upu_value ==> NOEUD_TENSION_BUS2VL/1.1
NETWORK__BUS____3_TN_Upu_value ==> NOEUD_TENSION_BUS3VL/1.1
NETWORK__BUS____4_TN_Upu_value ==> NOEUD_TENSION_BUS4VL/1.1
NETWORK__BUS____5_TN_Upu_value ==> NOEUD_TENSION_BUS5VL/1.1
NETWORK__BUS____6_TN_Upu_value ==> NOEUD_TENSION_BUS6VL/1.1
NETWORK__BUS____7_TN_Upu_value ==> NOEUD_TENSION_BUS7VL/1.1
NETWORK__BUS____8_TN_Upu_value ==> NOEUD_TENSION_BUS8VL/1.1
NETWORK__BUS____9_TN_Upu_value ==> NOEUD_TENSION_BUS9VL/1.1
NETWORK__BUS___10_TN_Upu_value ==> NOEUD_TENSION_BUS 10VL/1.1
NETWORK__BUS___11_TN_Upu_value ==> NOEUD_TENSION_BUS 11VL/1.1
NETWORK__BUS___12_TN_Upu_value ==> NOEUD_TENSION_BUS 12VL/1.1
NETWORK__BUS___13_TN_Upu_value ==> NOEUD_TENSION_BUS 13VL/1.1
NETWORK__BUS___14_TN_Upu_value ==> NOEUD_TENSION_BUS 14VL/1.1
GEN____1_SM_generator_PGen ==> GRP_PUISS_ACT_GEN1
GEN____1_SM_generator_QGen ==> GRP_PUISS_REA_GEN1
GEN____1_SM_generator_UStatorPu ==> 

In [11]:
# Build a joint dataframe to show comparisons:
comp_df = pd.DataFrame(columns=['Dynawo_variable', 'Dynawo_result', 'Astre_result', 'Astre_variable'])

for i in range(len(dydata['variable'])):
    vd = dydata['variable'].iat[i]
    va = vardict.loc[vardict['Dynawo'] == vd, ['Astre']]
    if (not va.empty and not va.isnull().values.any()):
        tmp1 = dydata[['variable', 'Dynawo_result']].iloc[i]
        tmp2 = asdata[['Astre_result', 'variable']].loc[asdata['variable'] == va.iat[0,0]]
        new_row = {'Dynawo_variable': tmp1[0], 'Dynawo_result': tmp1[1],
                   'Astre_result': tmp2.iat[0,0], 'Astre_variable': tmp2.iat[0,1]}
        comp_df = comp_df.append(new_row, ignore_index=True)


In [12]:
# Show it
sparklines.show(comp_df)

In [13]:
# Output this same thing to a standalone html file
sparklines.to_html(comp_df, html_output_file)
